In [22]:
import geopandas as gpd
import folium
import pandas as pd

# Load GeoJSON files
bus_stops = gpd.read_file("Bus Stop.geojson")
land_use = gpd.read_file("Landuse.geojson")
study_area = gpd.read_file("Study Area.geojson")
roads = gpd.read_file("Road.geojson")

# Ensure all are in the same CRS (WGS84)
bus_stops = bus_stops.to_crs(epsg=4326)
land_use = land_use.to_crs(epsg=4326)
study_area = study_area.to_crs(epsg=4326)
roads = roads.to_crs(epsg=4326)

print("✅ GeoJSON files loaded successfully.")

# Reproject bus stops to projected CRS for accurate buffer (400m)
bus_stops_proj = bus_stops.to_crs(epsg=32647)
bus_stop_buffer_proj = bus_stops_proj.buffer(400)
bus_stop_buffer = gpd.GeoDataFrame(geometry=bus_stop_buffer_proj, crs="EPSG:32647").to_crs(epsg=4326)

# Clip land use with buffer
land_use_clipped = gpd.overlay(land_use, bus_stop_buffer, how='intersection')

# Rename land use column if needed
if 'land_use_type' not in land_use_clipped.columns:
    land_use_clipped.rename(columns={'gtn1': 'land_use_type'}, inplace=True)

land_use_counts = land_use_clipped['land_use_type'].value_counts()
print("\n✅ Land Use Summary (within 400m buffer):")
print(land_use_counts)

# Clip study area to buffer and calculate population
study_area_clipped = gpd.overlay(study_area, bus_stop_buffer, how='intersection')
total_population = study_area_clipped['population'].sum() if 'population' in study_area_clipped.columns else 0
print(f"\n✅ Total population within 400m of a bus stop: {total_population}")

# ✅ Fix centroid warning by using projected CRS
study_area_proj = study_area.to_crs(epsg=32647)
centroid = study_area_proj.geometry.centroid.unary_union.centroid
centroid_wgs = gpd.GeoSeries([centroid], crs="EPSG:32647").to_crs(epsg=4326).geometry[0]
map_center = [centroid_wgs.y, centroid_wgs.x]

# Initialize folium map
m = folium.Map(location=map_center, zoom_start=13)

# Style study area based on 'layer'
def style_study_area(feature):
    color = '#3186cc' if feature['properties']['layer'] == 'Internal' else '#ff7800'
    return {'fillColor': color, 'color': color, 'weight': 1.5, 'fillOpacity': 0.2}

folium.GeoJson(
    study_area,
    name="Study Area",
    style_function=style_study_area,
    tooltip=folium.GeoJsonTooltip(fields=['layer'])
).add_to(m)

# Add bus stops
for idx, row in bus_stops.iterrows():
    if row.geometry:
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=4,
            color='black',
            fill=True,
            fill_color='yellow',
            popup=f"Bus Stop {idx+1}"
        ).add_to(m)

# Add buffer zone
folium.GeoJson(
    bus_stop_buffer,
    name="Bus Stop Buffer (400m)",
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'fillOpacity': 0.1}
).add_to(m)

# Add clipped land use
folium.GeoJson(
    land_use_clipped,
    name="Land Use in Buffer",
    style_function=lambda x: {'color': 'green', 'fillOpacity': 0.3}
).add_to(m)

# Add roads
folium.GeoJson(
    roads,
    name="Roads",
    style_function=lambda x: {'color': 'gray', 'weight': 1.2}
).add_to(m)

# ✅ KEEP LEGEND
legend_html = """
     <div style="position: fixed; 
                 bottom: 30px; left: 30px; width: 260px; height: 160px; 
                 background-color: white; z-index:9999; font-size:14px;
                 border:2px solid grey; padding: 10px;">
     <b>Legend</b><br>
     <i style="background:red;opacity:0.5;width:12px;height:12px;display:inline-block;"></i> Bus Stop Buffer (400m)<br>
     <i style="background:green;opacity:0.5;width:12px;height:12px;display:inline-block;"></i> Land Use in Buffer<br>
     <i style="background:gray;width:12px;height:2px;display:inline-block;"></i> Roads<br>
     <i style="background:#3186cc;opacity:0.5;width:12px;height:12px;display:inline-block;"></i> Study Area - Internal<br>
     <i style="background:#ff7800;opacity:0.5;width:12px;height:12px;display:inline-block;"></i> Study Area - External<br>
     </div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# ✅ ADD TITLE
title_html = '''
     <h3 align="center" style="font-size:20px"><b>Public Transport Accessibility and Land Use Analysis (400m Bus Stop Buffer)</b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add layer control
folium.LayerControl().add_to(m)

# Save map
m.save("buffer_analysis_map3.html")
print("\n✅ Interactive map saved as 'buffer_analysis_map3.html'")


C:\Users\ANY2024\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 1 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


✅ GeoJSON files loaded successfully.

✅ Land Use Summary (within 400m buffer):
land_use_type
Perumahan                             113254
Komersial                              24346
Industri                               10801
Tanah Kosong                            3741
Tanah Lapang dan Rekreasi               2972
Infrastruktur dan Utiliti               2378
Institusi dan Kemudahan Masyarakat       727
Pengangkutan                             448
Badan Air                                120
Pertanian                                 70
Hutan                                      2
Name: count, dtype: int64

✅ Total population within 400m of a bus stop: 228012183


C:\Users\ANY2024\AppData\Local\Temp\ipykernel_85364\2020630635.py:42: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = study_area_proj.geometry.centroid.unary_union.centroid



✅ Interactive map saved as 'buffer_analysis_map3.html'
